In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import zipfile

In [5]:
in_fp = '../data/raw/application_data.csv.zip'

zf = zipfile.ZipFile(in_fp) 
loan = pd.read_csv(zf.open('application_data.csv'))

In [6]:
loan

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


### Drop the columns that have > 35% missing values and are not relevant to our classification task.

In [4]:
loan = loan.drop(['OWN_CAR_AGE','EXT_SOURCE_1','APARTMENTS_AVG','BASEMENTAREA_AVG','YEARS_BEGINEXPLUATATION_AVG','YEARS_BUILD_AVG','COMMONAREA_AVG','ELEVATORS_AVG','ENTRANCES_AVG','FLOORSMAX_AVG','FLOORSMIN_AVG','LANDAREA_AVG','LIVINGAPARTMENTS_AVG','LIVINGAREA_AVG','NONLIVINGAPARTMENTS_AVG','NONLIVINGAREA_AVG','APARTMENTS_MODE','BASEMENTAREA_MODE','YEARS_BEGINEXPLUATATION_MODE','YEARS_BUILD_MODE','COMMONAREA_MODE','ELEVATORS_MODE','ENTRANCES_MODE','FLOORSMAX_MODE','FLOORSMIN_MODE','LANDAREA_MODE','LIVINGAPARTMENTS_MODE','LIVINGAREA_MODE','NONLIVINGAPARTMENTS_MODE','NONLIVINGAREA_MODE','APARTMENTS_MEDI','BASEMENTAREA_MEDI','YEARS_BEGINEXPLUATATION_MEDI','YEARS_BUILD_MEDI','COMMONAREA_MEDI','ELEVATORS_MEDI','ENTRANCES_MEDI','FLOORSMAX_MEDI','FLOORSMIN_MEDI','LANDAREA_MEDI','LIVINGAPARTMENTS_MEDI','LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI','HOUSETYPE_MODE','TOTALAREA_MODE','WALLSMATERIAL_MODE','FONDKAPREMONT_MODE','EMERGENCYSTATE_MODE'], axis = 1)

### OHE

In [6]:
loan_ohe_features = ['NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','WEEKDAY_APPR_PROCESS_START','OCCUPATION_TYPE','ORGANIZATION_TYPE']

There are 12 features that need to be one hot encoded.


In [7]:
for i in loan_ohe_features:
    loan[i] = i + ': ' + loan[i].astype(str)

In [8]:
loan_enc = OneHotEncoder(handle_unknown='ignore')
loan_enc.fit(loan[loan_ohe_features])

OneHotEncoder(handle_unknown='ignore')

In [10]:
loan_ohe_features_result = loan_enc.transform(loan[loan_ohe_features]).toarray()

### Create feature array and name it as loan_x

In [11]:
loan_x = np.append(loan_ohe_features_result, loan.drop(loan_ohe_features, axis = 1).drop('TARGET', axis = 1).values, axis = 1)

In [43]:
ohe_column_names = np.concatenate(loan_enc.categories_).tolist()
other_column_names = loan.drop(loan_ohe_features, axis = 1).drop('TARGET', axis = 1).columns
column_names = ohe_column_names + other_column_names.tolist()

### Create target array and name it as loan_y

In [14]:
loan_y = loan['TARGET']

0    282686
1     24825
Name: TARGET, dtype: int64

## Create df

In [57]:
all_columns = loan_y.tolist() + loan_x.tolist()

In [63]:
df_x = pd.DataFrame(loan_x, columns = column_names)

In [64]:
df_x['Target'] = loan_y

In [67]:
feature_engineered_loan_data = df_x

### Impute missing values

In [122]:
fill_mean = ['AMT_ANNUITY','AMT_GOODS_PRICE','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_LAST_PHONE_CHANGE']
fill_mode = ['CNT_FAM_MEMBERS', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE','AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR']

In [ ]:
for i in fill_mean:
    feature_engineered_loan_data[i].fillna(int(feature_engineered_loan_data[i].mean()), inplace=True)
for i in fill_mode:
    feature_engineered_loan_data[i].fillna(int(feature_engineered_loan_data[i].mode()), inplace=True)

In [126]:
feature_engineered_loan_data.to_csv('feature_engineered_loan_data.csv', index = False)

In [128]:
haha = pd.read_csv('feature_engineered_loan_data.csv')